# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re



## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
display(df)
df['Результат анализа'] = np.where(df['Результат анализа'] == 'б/цвета','0',df['Результат анализа'])

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [281]:
df['normativ'] = df['Норматив'].str.split(' ').apply(lambda x: x[2])
df['normativ1'] = df['normativ'].str.split('-').apply(lambda x : x[0]).str.replace(',','.')                      
df['normativ2'] = (df['normativ'] + '-0').str.split('-').apply(lambda x : x[1]).str.replace(',','.') 

df['res'] = np.select(
    [df['Норматив'].str.contains('не более'),
     df['Норматив'].str.contains('в пределах')],
    
        [np.where(
        df['Результат анализа'].apply(lambda x: float(x)) <= df['normativ1'].apply(lambda x: float(x)),
        'True',
        'False'),
         
        np.where(
        ( (df['Результат анализа'].apply(lambda x: float(x)) >= df['normativ1'].apply(lambda x: float(x)))
        & (df['Результат анализа'].apply(lambda x: float(x)) <= df['normativ2'].apply(lambda x: float(x))) ),
        'True',
        'False')
        ],   
)
df

,Показатель,Единица измерений,Результат анализа,Норматив,normativ,normativ1,normativ2,res
0,pH,единицы pH,8.4,в пределах 6-9,6-9,6,9,True
1,Запах,баллы,1,не более 2-3,2-3,2,3,True
2,Цветность,градусы,0,не более 30,30,30,0,True
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,7-10,7,10,True
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5","1,5",1.5,0,True
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3","3,3",3.3,0,True
6,Нитраты (по NO3),мг/дм3,24,не более 45,45,45,0,True
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5","3,5",3.5,0,True
8,Хлориды (Cl),мг/дм3,200,не более 350,350,350,0,True
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,500,500,0,True


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [3]:
conditions = ['orange', 'apple']
N = 1
i = 0
res = []

while i < 100000:

    count_orange = 5
    count_fruits = 9
    
    p_orange = count_orange / count_fruits

    fruit1 = np.random.choice(conditions, N, p=[p_orange, 1 - p_orange])
    if fruit1[0] == 'orange':
        count_orange = count_orange - 1
    count_fruits = count_fruits - 1
    p_orange = count_orange / count_fruits
    
    fruit2 = np.random.choice(conditions, N, p=[p_orange, 1 - p_orange])
    if fruit2[0] == 'orange':
        count_orange = count_orange - 1       
    count_fruits = count_fruits - 1
    p_orange = count_orange / count_fruits

    fruit3 = np.random.choice(conditions, N, p=[p_orange, 1 - p_orange])
    if fruit1[0] == fruit2[0] == fruit3[0] == 'orange':
        res.append(1)
    else:
        res.append(0)
    i = i + 1
    
sum(res)/len(res)

0.11816

In [4]:
conditions = ['orange', 'apple','orange','apple','orange','apple','orange','apple','orange']

friuts = np.random.choice(conditions, 3, replace=False)
friuts

friuts = list(map(lambda x:
                  np.random.choice(conditions, 3, replace=False),
                  range(1000000)
                 ))

df = pd.DataFrame({'res': friuts})
df['res1'] = df['res'].apply(lambda x:x[0])
df['res2'] = df['res'].apply(lambda x:x[1])
df['res3'] = df['res'].apply(lambda x:x[2])
print(df.query("res1 == 'orange' & res2 == 'orange' & res3 == 'orange'").shape[0]/df.shape[0])

0.119247


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [5]:
# Ваше решение
conditions = ['error', 'standart']
N = 1
i = 0
res = []

while i < 100000:

    count_error = 3
    count_details = 10
    detail = 'error'
    
    j = 0
    while detail[0] != 'standart':
        p_error = count_error / count_details
        detail = np.random.choice(conditions, N, p=[p_error, 1 - p_error])
        if detail[0] == 'error':
            count_error = count_error - 1
        count_details = count_details - 1
        j = j + 1
    if j == 2:
        res.append(1)
    else:
        res.append(0)
    i = i + 1

sum(res)/len(res)

0.23589

In [6]:
conditions = ['standart','standart','standart','error','standart','standart','error','standart','error','standart']

details = list(map(lambda x:
                  np.random.choice(conditions, 2, replace=False),
                  range(1000000)
                 ))
df = pd.DataFrame({'res': details})
df['res1'] = df['res'].apply(lambda x:x[0])
df['res2'] = df['res'].apply(lambda x:x[1])

print(df.query("res1 == 'error' & res2 == 'standart'").shape[0]/df.shape[0])

0.23442
